In [1]:
pip install statsmodels

     |████████████████████████████████| 8.1MB 5.5MB/s eta 0:00:01     |████████▌                       | 2.1MB 5.5MB/s eta 0:00:02
     |████████████████████████████████| 235kB 24.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np
import pandas as pd

In [24]:
hept=pd.read_csv('hepatitis1B.csv').drop(['1.3'],axis=1)

In [30]:
nobs=1
x_train,x_test=hept[:-nobs],hept[-nobs:]
hept.describe()

,2,30,2.1,1,2.2,2.3,2.4,2.5,1.1,2.6,2.7,2.8,2.9,2.10,1.2,85,18,4,35
count,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.00000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000
mean,1.790541,41.162162,1.094595,1.500000,1.837838,1.351351,1.614865,1.797297,1.77027,1.513514,1.783784,1.655405,1.858108,1.864865,1.493243,103.331081,230.885135,3.729730,51.202703
std,0.408305,12.366321,0.293648,0.515079,0.369851,0.479014,0.488280,0.403378,0.49617,0.577191,0.429214,0.490908,0.369042,0.362318,1.220553,47.082413,1524.617505,0.665931,21.649326
min,1.000000,7.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.000000,14.000000,2.000000,21.000000
25%,2.000000,32.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.00000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,78.000000,34.000000,3.000000,35.000000
50%,2.000000,39.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,93.000000,60.000000,4.000000,38.500000
75%,2.000000,50.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,120.000000,120.750000,4.000000,66.000000
max,2.000000,78.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,8.000000,295.000000,18585.000000,6.000000,100.000000


In [26]:
from statsmodels.tsa.stattools import grangercausalitytests

In [27]:
maxlag=2
test='ssr_chi2test'

In [28]:
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):
    X_train = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in X_train.columns:
        for r in X_train.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y={r},X={c},P Values = {p_values}')
            min_p_value=np.min(p_values)
            X_train.loc[r, c] = min_p_value
    X_train.columns =[var + '_x' for var in variables]
    X_train.index =  [var + '_y' for var in variables]
    return X_train

In [29]:
grangers_causation_matrix(x_train,variables=x_train.columns)

,2_x,30_x,2.1_x,1_x,2.2_x,2.3_x,2.4_x,2.5_x,1.1_x,2.6_x,2.7_x,2.8_x,2.9_x,2.10_x,1.2_x,85_x,18_x,4_x,35_x
2_y,1.0000,0.4805,0.5583,0.6298,0.3011,0.6173,0.7151,0.5075,0.9304,0.3469,0.2692,0.4820,0.2992,0.0214,0.4501,0.0203,0.0714,0.9498,0.1851
30_y,0.1437,1.0000,0.1286,0.2858,0.2867,0.6414,0.3046,0.2315,0.6612,0.5741,0.5253,0.7674,0.6226,0.9406,0.0523,0.0689,0.1959,0.2553,0.4963
2.1_y,0.5105,0.2202,1.0000,0.5875,0.1816,0.5634,0.3688,0.3300,0.0317,0.1230,0.4513,0.6533,0.1245,0.1372,0.1002,0.7249,0.6681,0.2465,0.2043
1_y,0.0173,0.0143,0.1626,1.0000,0.5031,0.0294,0.1718,0.8368,0.1399,0.1481,0.5559,0.7999,0.2071,0.7838,0.4841,0.1268,0.3430,0.4632,0.4950
2.2_y,0.6640,0.2001,0.0739,0.3431,1.0000,0.5194,0.5690,0.2952,0.8424,0.6536,0.3175,0.0580,0.4856,0.2103,0.2489,0.9341,0.0184,0.1611,0.3698
2.3_y,0.1175,0.3518,0.6233,0.6017,0.1077,1.0000,0.4871,0.5794,0.3459,0.4080,0.0922,0.9270,0.2870,0.7160,0.3137,0.3398,0.6629,0.3761,0.1517
2.4_y,0.2294,0.6909,0.5336,0.1932,0.7775,0.6697,1.0000,0.2704,0.5223,0.2253,0.0303,0.6318,0.2132,0.3824,0.6274,0.0658,0.2720,0.2283,0.4777
2.5_y,0.0014,0.0595,0.8038,0.8380,0.2716,0.5762,0.5512,1.0000,0.4881,0.9250,0.0481,0.2756,0.6133,0.8379,0.2473,0.6403,0.5149,0.2323,0.9630
1.1_y,0.1009,0.2086,0.8530,0.5224,0.3967,0.3935,0.6286,0.3212,1.0000,0.3139,0.3926,0.2014,0.3004,0.0047,0.2149,0.0020,0.2759,0.3417,0.8080
2.6_y,0.0261,0.7684,0.4116,0.8468,0.2175,0.4267,0.1931,0.0665,0.5157,1.0000,0.4613,0.0194,0.8206,0.0001,0.0171,0.0282,0.4790,0.4465,0.5581
